In [1]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [7]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Pre procesado de datos

Codificar datos categóricos

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder() #trasnsformar la columna 1 en dummies
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #ajustar y corregir automáticamente los valores que hay en columna 1
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [1])],   
    remainder='passthrough'                        
) #pasar los 1,2 y 3 hace columna dummies para columna 1
X = onehotencoder.fit_transform(X) #eliminar una columna dummie para no caer en la trampa de multicolinealidad
X = X[:, 1:]

In [9]:
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

Dividir en entrenamiento y testing

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Escalado de variables (cuando hay variables que destacan del resto, son más grandes)

In [12]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [13]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

Contruir red neuronal

In [14]:
import keras
from keras.models import Sequential #inicializar paámetros de la red
from keras.layers import Dense #declarar y crear capas intermedias

Iniciar red

In [15]:
classifier = Sequential()

Capas de entrada y primera capa oculta

In [19]:
#11 capas de entrada que son las variables independientes
classifier.add(Dense(units = 6,kernel_initializer='uniform',activation='relu', input_dim = 11)) 

In [20]:
#añadir segunda capa oculta
classifier.add(Dense(units = 6,kernel_initializer='uniform',activation='relu')) 

In [21]:
#capa de salida
classifier.add(Dense(units = 1,kernel_initializer='uniform',activation='sigmoid'))

Compilar la red

In [23]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics= ['accuracy'])

Evaluar modelo y calcular predicciones

In [24]:
classifier.fit(X_train, y_train, batch_size=10,epochs= 100)# batch es de cuanto en cuanto corregir

Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.4774 - accuracy: 0.7960
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4183 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4057 - accuracy: 0.8245
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3942 - accuracy: 0.8288
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3851 - accuracy: 0.8367
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3791 - accuracy: 0.8406
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3739 - accuracy: 0.8441
Epoch 8/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3707 - accuracy: 0.8457
Epoch 9/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3673 - accuracy: 0.8482
Epoch 10/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3642 - accura

Resultados Finales

In [25]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

Matriz confusión

In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [27]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [28]:
cm

array([[1516,   79],
       [ 210,  195]])

Precisión

In [29]:
(1516+195)/2000

0.8555